In [ ]:

"""
Versions:
Python @ 3.8.9
Scikit-Learn @ 1.0.2
Numpy @ 1.21.5
Matplotlib @ 3.5.1
Tensorflow @ 2.8.0

"""

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
# import processing

In [ ]:
people0_place1 = np.array(np.load("datasets/people0_place1.npy"))
people1_place1 = np.array(np.load("datasets/people1_place1.npy"))
people2_place1 = np.array(np.load("datasets/people2_place1.npy"))
people3_place1 = np.array(np.load("datasets/people3_place1.npy"))
people0_place2 = np.array(np.load("datasets/people0_place2.npy"))
people1_place2 = np.array(np.load("datasets/people1_place2.npy"))
people2_place2 = np.array(np.load("datasets/people2_place2.npy"))
people3_place2 = np.array(np.load("datasets/people3_place2.npy"))
people2_nonblocking = np.array(np.load("datasets/people2_nonblocking.npy"))
people3_nonblocking = np.array(np.load("datasets/people3_nonblocking.npy"))

data = np.concatenate((people0_place1,people1_place1,people2_place1,people3_place1[1000:], people3_nonblocking[1000:]))

print(data.shape)

In [ ]:
"""
Min-Max scaling in order to amplify the higher values from the acquistion.
"""
scaler = MinMaxScaler()
for i in range(len(data)):
    for j in range(3):
        data[i][j] = scaler.fit_transform(data[i][j])

In [ ]:
"""
Create labels.
"""
num_classes = 4
labels = np.zeros((len(data),1))
for i in range(num_classes-1):
    labels[(i+1)*int((len(data)/4)):] += 1


In [ ]:
"""
One hot encoding the labels.
"""
labels = tf.keras.utils.to_categorical(labels, num_classes=4, dtype='float32')

In [ ]:
range_doppler_map = processing.processing_rangeDopplerData(data)

In [ ]:

"""
Reshaping the data, thus the dimensionality reduction on the Convolution Layer happens on the Chirp and Sample dimensions.
"""
data = data.reshape(len(data),64,128,3)

In [ ]:
"""
Splitting the data.
"""
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.20, random_state = 34)

In [ ]:

print("x _train data shape: ",x_train.shape)
print("y_train data shape: ",y_train.shape)
print("x_test data shape: ",x_test.shape)
print("y test data shape: ",y_test.shape)

In [19]:


model = keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(64,128,3),padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='linear',padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='linear',padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))                  
model.add(layers.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(layers.Dropout(0.4))


model.add(layers.Flatten())
model.add(layers.Dense(128, activation='linear'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='linear'))
model.add(layers.LeakyReLU(alpha=0.1))               
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

#fitting the data into the model
model.fit(x_train,y_train,batch_size=20, epochs=20,validation_data=(x_test,y_test),verbose=1)



Epoch 1/20
320/320 [==============================] - 12s 33ms/step - loss: 1.0000 - accuracy: 0.5180 - val_loss: 0.6987 - val_accuracy: 0.6637
Epoch 2/20
320/320 [==============================] - 9s 29ms/step - loss: 0.7028 - accuracy: 0.6667 - val_loss: 0.6402 - val_accuracy: 0.6712
Epoch 3/20
320/320 [==============================] - 9s 29ms/step - loss: 0.6234 - accuracy: 0.7194 - val_loss: 0.5446 - val_accuracy: 0.7412
Epoch 4/20
320/320 [==============================] - 9s 29ms/step - loss: 0.5169 - accuracy: 0.7616 - val_loss: 0.4366 - val_accuracy: 0.7937
Epoch 5/20
320/320 [==============================] - 9s 29ms/step - loss: 0.4710 - accuracy: 0.7837 - val_loss: 0.5192 - val_accuracy: 0.7694
Epoch 6/20
320/320 [==============================] - 9s 29ms/step - loss: 0.4197 - accuracy: 0.8100 - val_loss: 0.4018 - val_accuracy: 0.8262
Epoch 7/20
320/320 [==============================] - 10s 30ms/step - loss: 0.3591 - accuracy: 0.8427 - val_loss: 0.3502 - val_accuracy: 0.83

In [22]:

y_pred = model.predict(x_test)

confusion_m = confusion_matrix(y_test.argmax(axis=-1), y_pred.argmax(axis=-1))
classification_rep = classification_report(y_test.argmax(axis=-1), y_pred.argmax(axis=-1))

print("Confusion Matrix\n", confusion_m)
print("Classification Report\n",classification_rep)

Confusion Matrix
 [[414   1   0   0]
 [  0 331  36   7]
 [  0 158 231   5]
 [  0  17   7 393]]
Classification Report
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       415
           1       0.65      0.89      0.75       374
           2       0.84      0.59      0.69       394
           3       0.97      0.94      0.96       417

    accuracy                           0.86      1600
   macro avg       0.87      0.85      0.85      1600
weighted avg       0.87      0.86      0.85      1600



In [21]:
"""
Saving the model.
"""
model.save("my_model_noblockin3_old_data.h5")